# DQN 
## Cartpole

Fabrice Mulotti


In [ ]:
import numpy as np
import gymnasium as gym
from collections import deque
import random

import tensorflow as tf
print(tf.__version__)

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam # si vous utilisz un mac M1/M2 utiliser legacy.optimizers

from IPython.display import display, clear_output
import matplotlib.pyplot as plt

%matplotlib inline

Si vous avez erreur TypeError: unhashable type: 'list', c'est que votre version de python est trop ancienne (conseillee 3.11)

In [ ]:
# tf.compat.v1.disable_eager_execution()

---
## Définition de l'environnement

In [ ]:
ENV_NAME = 'CartPole-v1'

env = gym.make(ENV_NAME)
np.random.seed(100)
env.reset(seed=100)
nb_actions = env.action_space.n # Nombre d'action
nb_obs = env.observation_space.shape[0] # nombre de paramètre pour décrire l'environnement
print(f"Nombre d actions : {nb_actions}, Dimension de l env : {nb_obs}")

# Remarques sur l'environnement

2 actions possibles :<br>
    poussée vers la gauche<br>
    poussée vers la droite<br>
    
Observation :<br>
    Valeurs **continues** !

## Fonctions utiles

In [ ]:
# Classe pour notre algorithme DQN
class DQN:
    def __init__(self, obs_size, action_size, gamma, learning_rate):
        # Objet : initialisation de la classe
        # Paramètres en entrée
        #     Nombre de paramètre pour décrire l'état
        #     Nombre d'actions
        #.    Paramètre gamma : dépréciation futur
        #     learning rate
        
        # dimension des états
        self.obs_size = obs_size
        
        # nombre d'actions
        self.action_size = action_size
        
        # gamma : dépréciation du futur
        self.gamma = gamma
        
        # learning rate
        self.learning_rate = learning_rate
        
        # autres paramètres
        self.update_freq = 100 # fréquence de copie des poids (1000)
        
        # autres structures
        self.replay_buffer = deque(maxlen=100000) # enregistrements des résultats
        
        # Modèle de réseaux de neurones
        self.main_network = self.build_model(16,8)
        self.target_network = self.build_model(16,8)
        # recopie des poids pour avoir des réseaux à l'identique
        self.target_network.set_weights(self.main_network.get_weights())
        
        self.debug = 0
        
    def build_model(self,layer1,layer2):
        # input : size of the 2 layers
        # output : model of neural network
        model = Sequential()
        model.add(Input(shape=(self.obs_size,)))
        model.add(Dense(layer1,activation='relu'))
        model.add(Dense(layer2,activation='relu'))
        model.add(Dense(self.action_size))
        model.add(Activation('linear'))
        model.compile(loss='mse',optimizer=Adam(learning_rate=self.learning_rate),jit_compile=True)
        return model

    def store_transition(self,state,action,reward,next_state,done):
        # input : 
        # output : nothing
        self.replay_buffer.append((state,action,reward,next_state,done))
             
    def policy(self, state, epsilon):
        # Policy : return action regarding the state according to the policy
        r = np.random.uniform()
        if r < epsilon:
            action = np.random.randint(self.action_size)
        else:
            # ############################################################################
            actions= # Votre code : prédiction
            action = # 
 
        return action
    
                       
    def train_model(self,batch_size):
        self.debug=0
        # sélection d'un échantillon
        minibatch = random.sample(self.replay_buffer,batch_size)
        states, actions, rewards, next_states, dones = map(np.array, zip(*minibatch))

        states=states.squeeze()
        next_states=next_states.squeeze()
        
        # On récupere les Q values des états rencontrés
        Q_values = self.main_network.predict(states,verbose = 0)
        
        Updates = rewards + self.gamma * np.amax(self.target_network.predict(next_states,verbose=0),axis=1) * (1-dones)
        
        Q_values[ np.arange(len(Updates)),actions] = Updates

        # ############################################################################
        # Votre code : entrainement
        # Astuce : vous pouvez mettre l'entrainement dans une fonction avec un décorateur tf.function
    
    def update_weights(self):
        self.target_network.set_weights(self.main_network.get_weights())

In [ ]:
# transformation des données pour compatabilité avec l'alimentation du réseau de neurones (1,s)`
def trans_state(s):
    return  np.reshape(s, [1, nb_obs])

In [ ]:
num_episodes = 2000
batch_size = 10 # K
gamma=0.98

# variation du epsilon de notre politique e-greedy
epsilon_max=1.00
epsilon_min=0.05
epsilon_decay=0.992 

learning_rate=0.0001 # 0.001

In [ ]:
dqn = DQN(env.observation_space.shape[0],env.action_space.n,gamma,learning_rate)

In [ ]:
time_step = 0  # comptage du nombre total de mouvement
histoReturn=[] # pour graphique sur historique récompense
epsilon=epsilon_max
checkpoint_path="cartpole-save.weights.h5"

fig = plt.figure()
ax = fig.add_subplot(111)

for i in range(num_episodes):
    # somme de la récompense total pour une cycle
    Return = 0
    
    # reset env et conversion state
    state = trans_state(env.reset()[0])
    done=False
    truncated=False
    while not(done or truncated):
            
        time_step += 1
        
        # copie des poids du réseau d'entrainement vers le réseau cible selon la fréquence choisie
        if time_step % dqn.update_freq == 0:
            dqn.update_weights()
        
        # sélection d'une action selon notre politique
        # votre code
 
        # jouer l'action
        # ############################################################################
        # Votre code - jouez l'action et récupërer les éléments
        # Changer le format de l'état obtenu (fonction dispo)
        # 2 lignes
        
        if truncated:
            done=True 

        # enregistrement pour replay
        dqn.store_transition(state, action, reward, next_state, done)
        
        # et shift d'état
        state = next_state
        
        # cumul du retour G
        Return += reward

        # Done ?
        if done or truncated:
            # affichage du résultat du cyle
            print('Episode: ',i, ',' 'Return', Return,', epsilon ',np.round(epsilon,2),', end=',truncated)
            # ou
            # mise à jour de notre graphique sur les gains tous les 10 cycles
            #if i % 10 == 0:
            #    ax.clear()
            #    ax.plot(histoReturn)
            #    display(fig)
            #    clear_output(wait=True)
    
            # on décremmente epsilon
            epsilon *= epsilon_decay
            epsilon = max(epsilon_min, epsilon)
            histoReturn.append(Return)
            # Sauvegarde des poids tous les 50 cycles
            if i % 50 == 0:
                dqn.main_network.save_weights(checkpoint_path)
            break
            
        # à partir de + batch-size on entraine le reseau
        if len(dqn.replay_buffer) > batch_size:
            # Votre code : entrainement du réseau


---
# Visualisons le résultat

In [ ]:
ENV_NAME = 'CartPole-v1'

env = gym.make(ENV_NAME,render_mode='human')
nb_actions = env.action_space.n # Nombre d'action
nb_obs = env.observation_space.shape[0] # nombre de paramètre pour décrire l'environnement


In [ ]:
# Pas besoin d'exécuter cette séquence si vous lancez le test après l'entrainement
dqn = DQN(env.observation_space.shape[0],env.action_space.n,0,0)
dqn.main_network.load_weights("cartpole-save.weights.h5")

In [ ]:
state=trans_state(env.reset()[0])
done = False
truncated=False
Gain=0
while not (done or truncated):
    action = dqn.policy(state,0)
    next_state, reward, done, truncated , _ = env.step(action)
    Gain += reward
    state=trans_state(next_state)
print(f"Gain {Gain}")